In [158]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib inline
import matplotlib.image as mpimg

In [2]:
train_dataset = h5py.File('train_catvnoncat.h5',"r")
test_dataset = h5py.File('test_catvnoncat.h5',"r")

In [3]:
train_X_orig = np.array(train_dataset["train_set_x"][:])
train_X_orig.shape


(209, 64, 64, 3)

In [4]:
train_Y_orig = np.array(train_dataset["train_set_y"][:])
train_Y_orig.shape

(209,)

In [5]:
test_X_orig = np.array(test_dataset["test_set_x"][:])
test_X_orig.shape

(50, 64, 64, 3)

In [6]:
test_Y_orig = np.array(test_dataset["test_set_y"][:])
test_Y_orig.shape

(50,)

In [7]:
classes = np.array(test_dataset["list_classes"][:])
classes

array(['non-cat', 'cat'], 
      dtype='|S7')

In [8]:
train_Y_orig = train_Y_orig.reshape(1,train_Y_orig.shape[0])
test_Y_orig = test_Y_orig.reshape(1,test_Y_orig.shape[0])


In [11]:
m_train = train_X_orig.shape[0]
m_train

209

In [12]:
m_test = test_X_orig.shape[0]
m_test

50

In [14]:
num_px = train_X_orig.shape[1]
num_px

64

In [17]:
train_X_orig = train_X_orig.reshape(m_train,num_px ** 2 * 3).T
train_X_orig.shape

(12288, 209)

In [18]:
test_X_orig = test_X_orig.reshape(m_test,num_px ** 2 * 3).T
test_X_orig.shape

(12288, 50)

In [22]:
train_Y_orig = train_Y_orig.reshape(1,-1)
train_Y_orig.shape

(1, 209)

In [23]:
test_Y_orig = test_Y_orig.reshape(1,-1)
test_Y_orig.shape

(1, 50)

In [142]:
train_X =1.0 * train_X_orig / 255
train_X.shape[1]

209

In [140]:
test_X = 1.0 * test_X_orig /255
test_X.shape

(12288, 50)

In [31]:
len(train_X)

12288

In [34]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [35]:
sigmoid(0)

0.5

In [39]:
sigmoid(np.array([0 ,100]))

array([ 0.5,  1. ])

In [141]:
w = np.zeros((num_px ** 2 * 3,1))
w.shape

(12288, 1)

In [44]:
b = 0

In [146]:
def propagate(X,y,w,b):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X) + b)
    J = np.dot(y,np.log(A.T)) + np.dot((1 - y),np.log((1 - A).T))
    J = - J * 1./ m
    dz = A - y
    db = (1./ m)* np.sum(dz)
    dw = (1./ m) * np.dot(X,dz.T)
    grads = {"dw": dw,
             "db": db}
    return grads,J

In [147]:
w, b, X, Y = np.array([[1],[2]]), 2, np.array([[1,2],[3,4]]), np.array([[1,0]])
grads, cost = propagate(X,Y,w,b)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))
#cost.shape

dw = [[ 0.99993216]
 [ 1.99980262]]
db = 0.499935230625
cost = [[ 6.00006477]]


In [148]:
def optimize(X,Y,w,b,num_iter,learning_rate):
    for i in range(num_iter):
        grads,cost = propagate(X,Y,w,b)
        w = w - learning_rate * grads["dw"]
        b = b - learning_rate * grads["db"]
    paras = {"w" : w,"b" : b}
    return paras,grads,cost

In [149]:
params, grads, costs = optimize(X,Y,w,b,num_iter= 100, learning_rate = 0.009)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))


w = [[ 0.1124579 ]
 [ 0.23106775]]
b = 1.55930492484
dw = [[ 0.90158428]
 [ 1.76250842]]
db = 0.430462071679


In [152]:
def predict(w,b,X):
    Y = sigmoid(np.dot(w.T,X) + b)
    m = Y.shape[1]
    for x in range(m):
        #print (Y[0][x])
        if(Y[0][x] > 0.5):
            Y[0][x] = 1
        else:
            Y[0][x] = 0
    return Y

In [153]:
print ("predictions = " + str(predict(w, b, X)))

predictions = [[ 1.  1.]]


In [155]:
def model(train_X,train_Y,test_X,test_Y,num_iter = 2000,learning_rate = 0.5):
    b = 0
    w = np.zeros((train_X.shape[0],1))
    paras,grads,cost = optimize(train_X,train_Y,w,b,num_iter,learning_rate)
    w = paras["w"]
    b = paras["b"]
    Y_prediction_test = predict(w,b,test_X)
    Y_prediction_train = predict(w,b,train_X)
    d = {"costs": cost,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : paras["w"], 
         "b" : paras["b"],
         "learning_rate" : learning_rate,
         "num_iterations": num_iter}
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - train_Y)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - test_Y)) * 100))
    return d

In [156]:
d = model(train_X,train_Y_orig,test_X,test_Y_orig,num_iter = 2000,learning_rate = 0.005)

train accuracy: 99.043062201 %
test accuracy: 70.0 %
